In [67]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
import pylab as plt
import logging
from sklearn.linear_model import LogisticRegression
import os
from os.path import expanduser
logging.basicConfig()
logger = logging.getLogger("wrapper_svm")
logger.setLevel(logging.INFO)
from aisdc.safemodel.classifiers import SafeSVC

### Normal model

In [68]:
imaging = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/imaging.csv')
imaging = imaging.drop(columns=['COLPROT','VISCODE','VISCODE2','EXAMDATE','VERSION','LONIUID',
                                  'IMAGEUID','RUNDATE','STATUS','OVERALLQC','TEMPQC','FRONTQC','PARQC',
                                  'INSULAQC','OCCQC','BGQC','CWMQC','VENTQC','HIPPOQC','update_stamp'])
imaging = imaging.set_index('RID')

imaging = imaging.filter(regex='CV')

depression = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/medical_hist.csv')
depression = depression.set_index('RID')
depression = pd.DataFrame(depression['IHDESC'])
depression = depression[depression['IHDESC'].str.lower().isin(['depression'])]

features = imaging.copy()
features = features.loc[~features.index.duplicated(keep='last')]
features['Depression'] = features.index.isin(depression.index).astype(int)

demographics = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/demog.csv')
demographics = demographics.set_index('RID')

demographics_features = demographics[['PTMARRY','PTEDUCAT','PTHOME','PTETHCAT','PTRACCAT']]

demographics['PTDOBYY'] = 2023 - demographics['PTDOBYY']
age = demographics['PTDOBYY']
age = pd.DataFrame(age)
age = age.loc[~age.index.duplicated(keep='last')]

gender = demographics['PTGENDER']
gender = pd.DataFrame(gender)
gender = gender.loc[~gender.index.duplicated(keep='last')]

features = features.join(age, how='inner')
features = features.join(gender, how='inner')

gds = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/gds.csv')  #geriatric depression scale (5-8 mild depression, 9-11 moderate depression)
gds = gds.set_index('RID')
gds = pd.DataFrame(gds['GDTOTAL'])
gds = gds.loc[~gds.index.duplicated(keep='last')]
features = features.join(gds, how='inner')

cdr = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/cdr.csv')  #cdr dementia score (0 = no, 2/3 = dementia)
cdr = cdr.set_index('RID')
cdr = pd.DataFrame(cdr['CDGLOBAL'])
cdr = cdr.loc[~cdr.index.duplicated(keep='last')]
features = features.join(cdr, how='inner')

freesurfer_names = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/FreeSurferNames.csv')
freesurfer_names = freesurfer_names[freesurfer_names['ID'].str.contains('CV')]
freesurfer_names['NAME'] = freesurfer_names['NAME'].apply(lambda x: x.split()[-1])
freesurfer_names = freesurfer_names.set_index('ID')
freesurfer_names = freesurfer_names.to_dict()

features.rename(columns=freesurfer_names['NAME'], inplace=True)

mmse = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/mmse.csv')
mmse = mmse.set_index('RID')
mmse = pd.DataFrame(mmse['MMSCORE'])
mmse = mmse.loc[~mmse.index.duplicated(keep='last')]
features = features.join(mmse, how='inner')

fhq = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/fhq.csv')
fhq = fhq.set_index('RID')
fhq = fhq[['FHQMOM','FHQDAD','FHQSIB']]
features = features.join(fhq, how='inner')

phq = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/phq.csv')
phq = phq.set_index('RID')
phq = phq[['PHS','CIR']]
features = features.join(phq, how='inner')

medhist = pd.read_csv('C:/Users/alieyeh.moghaddam/Desktop/ADNI/medhist.csv')
medhist = medhist.set_index('RID')
medhist = medhist[['MHPSYCH','MH2NEURL','MH3HEAD','MH4CARD','MH5RESP','MH6HEPAT','MH7DERM','MH8MUSCL','MH9ENDO','MH10GAST','MH11HEMA','MH12RENA','MH13ALLE','MH14ALCH','MH15DRUG','MH16SMOK','MH17MALI']]
medhist = medhist.loc[~medhist.index.duplicated(keep='last')]
features = features.join(medhist, how='inner')
"""
"""

features['Dementia'] = ((features['CDGLOBAL'] > 0.6)).astype(int)
features = features.drop(columns='GDTOTAL')
features = features.drop(columns='CDGLOBAL')

In [69]:
features['Dementia'].value_counts()

0    307
1     59
Name: Dementia, dtype: int64

In [70]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

X = features.drop(columns='Dementia')
names = X.columns.tolist()
y = features['Dementia']

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp.fit_transform(X)

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

X = pd.DataFrame(X, columns=names)
y = pd.DataFrame(y)

In [71]:
space={'C': hp.uniform('C', 0, 20),
       # 'scale': hp.choice('scale', [0, 1]),
       # 'normalize': hp.choice('normalize', [0, 1]),
       'degree':hp.choice('degree',[2,3,4]),
       'gamma': hp.uniform('gamma', 0, 20)
       # 'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
       #,'k' : hp.quniform('k', 10, 280,1)
    }

In [72]:
def objective2(space):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    svc = SVC(**space, kernel='rbf',probability=True)
    
    svc.fit(X_train, np.ravel(y_train,order="c"))
    
    pred = svc.predict(X_test)
    #accuracy = accuracy_score(y_test, pred>0.5)
    f = f1_score(y_test, pred) #, average='macro'
    #print ("SCORE:", f)
    return {'loss': -f, 'status': STATUS_OK }

In [73]:
trials = Trials()

best_hyperparams = fmin(fn = objective2,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 10,
                        trials = trials)

100%|██████████████████████████████████████████████| 10/10 [00:01<00:00,  7.60trial/s, best loss: -0.16417910447761194]


In [74]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'C': 1.4793665528429822, 'degree': 1, 'gamma': 0.07845532987260873}


In [75]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp.fit_transform(X_train_orig)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_test = imp.fit_transform(X_test_orig)
    
model = SVC(**best_hyperparams, kernel='rbf', probability=True)
    
model.fit(X_train, np.ravel(y_train,order="c"))
    
predictions = model.predict(X_test)
predict_porb = model.predict_proba(X_test)


In [76]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [79]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred)*100)
print("Precision:", metrics.precision_score(y_test, y_pred)*100)
print("Recall:", metrics.recall_score(y_test, y_pred)*100)
print("F1-score:", metrics.f1_score(y_test, y_pred)*100)

Accuracy: 48.78048780487805
Precision: 48.760330578512395
Recall: 98.33333333333333
F1-score: 65.19337016574585


In [78]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
confusion_matrix

array([[ 1, 62],
       [ 1, 59]], dtype=int64)

In [38]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Safe Model

In [89]:
# dhat = 5  # Dimension of approximator
# eps = 500  # DP level (not very private)
C = best_hyperparams['C']
gamma = best_hyperparams['gamma']

In [90]:
clf3 = SafeSVC(C=C, gamma=gamma)
clf3.fit(X_train, np.ravel(y_train,order="c"))
c3 = clf3.predict(X_test)
p3 = clf3.predict_proba(X_test)

In [46]:
clf3.save(name="testSaveSVC.pkl")
clf3.request_release(path="testSaveSVC", ext="pkl")
target_json = os.path.normpath("testSaveSVC/target.json")
with open(target_json, "r") as f:
    print(f.read())

{
    "data_name": "",
    "n_samples": 0,
    "features": {},
    "n_features": 0,
    "n_samples_orig": 0,
    "generalisation_error": "unknown",
    "safemodel": [
        {
            "researcher": "alieyeh.moghaddam",
            "model_type": "SVC",
            "details": "Model parameters are within recommended ranges.\n",
            "recommendation": "Proceed to next step of checking",
            "timestamp": "2023-12-04 09:56:45"
        }
    ],
    "model_path": "model.pkl",
    "model_name": "SafeSVC",
    "model_params": {}
}


In [91]:
print("Accuracy:", metrics.accuracy_score(y_test, c3)*100)
print("Precision:", metrics.precision_score(y_test, c3)*100)
print("Recall:", metrics.recall_score(y_test, c3)*100)
print("F1-score:", metrics.f1_score(y_test, c3))

Accuracy: 47.96747967479675
Precision: 46.96969696969697
Recall: 51.66666666666667
F1-score: 0.4920634920634921


### Attacks 

In [ ]:
# !git clone https://github.com/AI-SDC/AI-SDC.git
# !pip install '/content/AI-SDC/'

In [80]:
import sys
import os
import pandas as pd
import numpy as np
import json
import random
from itertools import product
from scipy.stats import poisson

from os.path import expanduser

ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath("")))
sys.path.append(ROOT_DIR)
home = expanduser("~")
sys.path.append(os.path.abspath(home + "/AI-SDC"))

np.random.seed(1234)
random.seed(12345)

from aisdc.attacks.multiple_attacks import (  # pylint: disable = import-error
    ConfigFile,
    MultipleAttacks,
)
from aisdc.attacks.target import Target  # pylint: disable = import-error




In [81]:
target = Target(model=model)
target.name = "adni_dem"
target.add_processed_data(X_train, y_train, X_test, y_test)
target.add_raw_data(X, y, X_train_orig, y_train_orig, X_test_orig, y_test_orig)
for i in range(len(names)):
    target.add_feature(X.columns[i], indices=None, encoding='int')

# [Researcher] Dump the target model and target data
target.save(path="target")

In [ ]:
configfile_obj = ConfigFile(
    filename="single_config.json",
)

# Adding three worst-case attack configuration dictionaries to the JSON file
config = {
    "n_reps": 10,
    "n_dummy_reps": 1,
    "p_thresh": 0.05,
    "test_prop": 0.5,
    "train_beta": 5,
    "test_beta": 2,
    "report_name": "worstcase_example1_report",
}
configfile_obj.add_config(config, "worst_case")

config = {
    "n_reps": 20,
    "n_dummy_reps": 1,
    "p_thresh": 0.05,
    "test_prop": 0.5,
    "train_beta": 5,
    "test_beta": 2,
    "report_name": "worstcase_example2_report",
}
configfile_obj.add_config(config, "worst_case")

config = {
    "n_reps": 10,
    "n_dummy_reps": 1,
    "p_thresh": 0.05,
    "test_prop": 0.5,
    "train_beta": 5,
    "test_beta": 2,
    "report_name": "worstcase_example3_report",
    "training_preds_filename": "train_preds.csv",
    "test_preds_filename": "test_preds.csv",
    "attack_metric_success_name": "P_HIGHER_AUC",
    "attack_metric_success_thresh": 0.05,
    "attack_metric_success_comp_type": "lte",
    "attack_metric_success_count_thresh": 2,
    "attack_fail_fast": True,
}
configfile_obj.add_config(config, "worst_case")

# Adding two lira attack configuration dictionaries to the JSON file
config = {
    "n_shadow_models": 100,
    "report_name": "lira_example1_report",
    "training_data_filename": "train_data.csv",
    "test_data_filename": "test_data.csv",
    "training_preds_filename": "train_preds.csv",
    "test_preds_filename": "test_preds.csv",
    "target_model": ["sklearn.ensemble", "SVM"],
    "target_model_hyp": {"min_samples_split": 2, "min_samples_leaf": 1},
}
configfile_obj.add_config(config, "lira")

config = {
    "n_shadow_models": 150,
    "report_name": "lira_example2_report",
    "shadow_models_fail_fast": True,
    "n_shadow_rows_confidences_min": 10,
    "training_data_filename": "train_data.csv",
    "test_data_filename": "test_data.csv",
    "training_preds_filename": "train_preds.csv",
    "test_preds_filename": "test_preds.csv",
    "target_model": ["sklearn.ensemble", "RandomForestClassifier"],
    "target_model_hyp": {"min_samples_split": 2, "min_samples_leaf": 1},
}
configfile_obj.add_config(config, "lira")

# Adding a lira JSON configuration file to a configuration file
# having multiple attack configurations
config = {
    "n_shadow_models": 120,
    "report_name": "lira_example3_report",
    "shadow_models_fail_fast": True,
    "n_shadow_rows_confidences_min": 10,
    "training_data_filename": "train_data.csv",
    "test_data_filename": "test_data.csv",
    "training_preds_filename": "train_preds.csv",
    "test_preds_filename": "test_preds.csv",
    "target_model": ["sklearn.ensemble", "SVM"],
    "target_model_hyp": {"min_samples_split": 2, "min_samples_leaf": 1},
}
with open("lira_config.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(config))
configfile_obj.add_config("lira_config.json", "lira")

# Adding an attribute inference attack configuration dictionary to the JSON file
config = {
    "n_cpu": 2,
    "report_name": "aia_exampl1_report",
}
configfile_obj.add_config(config, "attribute")

# Instantiating MultipleAttacks object specifying a single configuration file
# (with multiple attacks configurations) and a single JSON output file
attack_obj = MultipleAttacks(
    config_filename="single_config.json",
    #output_filename="single_output_progammatic.json",
)
attack_obj.attack(target)

# [TRE] Runs the attack. This would be done on the command line, here we do that with os.system
# [TRE] First they access the help to work out which parameters they need to set
os.system(
    f"{sys.executable} -m aisdc.attacks.multiple_attacks run-attack-from-configfile --help"
)

# # [TRE] Then they run the attack
os.system(
    f"{sys.executable} -m aisdc.attacks.multiple_attacks run-attack-from-configfile "
    "--attack-config-json-file-name single_config.json "
    "--attack-target-folder-path target "
    "--attack-output-json-file-name single_output_cmd.json "
)